In [203]:
import pandas as pd
import os
from tqdm import tqdm
import seaborn as sns
import plotly.express as px
import math

## Tag extraction

In [11]:
# get file paths

directory = 'data/transcripts'
filepaths_list = []
transcripts_list = []

# iterate over files in that directory
for filename in os.listdir(directory):
    file = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(file) and file.endswith('.csv'):
        filepaths_list.append(file)

# sort transcripts chronologically 
filepaths_list.sort()
print(len(filepaths_list))

117


In [487]:
transcript_df = pd.DataFrame()

for filepath in tqdm(filepaths_list[:]):
    with open(filepath, encoding='utf-8') as file:
        session_df = pd.read_csv(file).drop(columns='Unnamed: 0')
        if session_df.iloc[0]['IdSession'] > 4000:
            transcript_df = pd.concat([transcript_df, session_df])

transcript_df = transcript_df.reset_index(drop=True)
transcript_df

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 117/117 [00:05<00:00, 21.70it/s]


,ID,IdSubject,PersonNumber,Text,MeetingCouncilAbbreviation,MeetingDate,IdSession,SpeakerFunction,CouncilId,Start,End,Function,LanguageOfText
0,3,1,498,"Madame la Présidente de la Confédération, \nSe...",N,19991206,4601,AP-M,1.0,1999-12-06T14:32:24,1999-12-06T14:33:39,NaN,FR
1,1,1,498,La séance d'aujourd'hui s'ouvre sous le signe ...,N,19991206,4601,AP-M,1.0,1999-12-06T14:36:50,1999-12-06T14:50:21,NaN,FR
2,53,6,498,Il appartenait au doyen d'âge de constituer le...,N,19991206,4601,AP-M,1.0,1999-12-06T14:50:21,1999-12-06T14:52:13,NaN,FR
3,52,6,273,Das provisorische Büro hat an seiner Sitzung v...,N,19991206,4601,NaN,1.0,1999-12-06T14:52:13,1999-12-06T15:01:02,b,DE
4,51,6,305,Le Bureau provisoire de notre Assemblée a siég...,N,19991206,4601,NaN,1.0,1999-12-06T15:01:02,1999-12-06T15:07:43,b,FR
...,...,...,...,...,...,...,...,...,...,...,...,...,...
189483,319686,60670,4238,"Sehr geschätzter Herr Nationalrat Egger, Sie h...",N,20230504,5120,BR-F,99.0,2023-05-04T17:04:38,2023-05-04T17:06:20,BR-F,DE
189484,319809,60674,4268,La questione della mediatizzazione dei process...,N,20230504,5120,Mit-F,1.0,2023-05-04T17:06:39,2023-05-04T17:11:27,Mit-F,IT
189485,319699,60674,1122,"Frau Kollegin Gysin, Sie wollen eine Priorisie...",N,20230504,5120,Mit-M,1.0,2023-05-04T17:11:29,2023-05-04T17:11:46,Mit-M,DE
189486,319807,60674,4268,"Collega Fluri, sono molto consapevole del prob...",N,20230504,5120,Mit-F,1.0,2023-05-04T17:11:46,2023-05-04T17:12:44,Mit-F,IT


In [488]:
with open('data/businesses_tags.csv', encoding='utf-8') as file:
    tags_df = pd.read_csv(file)
    tags_df = tags_df.rename(columns={'Unnamed: 0': 'BusinessNumber'})

tags_list = list(tags_df.columns)
tags_list.remove('BusinessNumber')
print(tags_list)
tags_df

['Éducation', 'Droit', 'Parlement', 'Économie', 'Politique internationale', 'Finances', 'Transports', 'Environnement', 'Santé', 'Politique européenne', 'Questions sociales', 'Politique de sécurité', 'Médias et communication', "Politique d'Etat", 'Science et recherche', 'Agriculture', 'Énergie', 'Aménagement du territoire et logement', 'Politique migratoire', 'Culture', 'Droit civil', 'Emploi et travail', "Droits de l'homme", 'Droit pénal', 'Droit international', 'Fiscalité', 'Protection sociale', 'Justice']


,BusinessNumber,Éducation,Droit,Parlement,Économie,Politique internationale,Finances,Transports,Environnement,Santé,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
0,19970419,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20000086,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20003605,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,20003606,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,20003647,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18189,20200002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
18190,20204399,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
18191,20210002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
18192,20210401,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [489]:
with open('data/subjects.csv', encoding='utf-8') as file:
    subjects_df = pd.read_csv(file).drop(columns='Unnamed: 0')
    
subjects_df

,IdSubject,BusinessNumber,BusinessShortNumber,Title,TitleFR
0,6688,19850019,85.0190,Utilisation pacifique de l'énergie nucléaire. ...,Utilisation pacifique\nde l'énergie nucléaire....
1,6905,19850019,85.0190,Utilisation pacifique de l'énergie nucléaire. ...,Utilisation pacifique\nde l'énergie nucléaire....
2,645,19850227,85.2270,Droit des assurances sociales,Initiative parlementaire\nMeier Josi.\nDroit d...
3,775,19850227,85.2270,Droit des assurances sociales,Initiative parlementaire\nMeier Josi.\nDroit d...
4,1234,19850227,85.2270,Droit des assurances sociales,Initiative parlementaire\nMeier Josi.\nDroit d...
...,...,...,...,...,...
28708,60010,20237245,23.7245,Pénurie de médicaments. Degré d’efficacité des...,Heure des questions.\nQuestion Feller Olivier....
28709,59975,20237252,23.7252,Le collaborateur de l'OFSP responsable de la g...,Heure des questions.\nQuestion Büchel Roland R...
28710,59999,20237253,23.7253,Chômage en hausse malgré la pénurie de main-d’...,Heure des questions.\nQuestion Strupler Manuel...
28711,59993,20237258,23.7258,Comment les recommandations du comité scientif...,Heure des questions.\nQuestion Python Valentin...


In [492]:
transcript_df = transcript_df[['ID', 'IdSubject', 'Text', 'PersonNumber', 'MeetingCouncilAbbreviation', 'MeetingDate', 'IdSession']]
transcript_subject_df = transcript_df.reset_index().merge(subjects_df[['IdSubject', 'BusinessNumber', 'Title', 'TitleFR']], on='IdSubject', how='left')
transcript_subject_df

,index,ID,IdSubject,Text,PersonNumber,MeetingCouncilAbbreviation,MeetingDate,IdSession,BusinessNumber,Title,TitleFR
0,0,3,1,"Madame la Présidente de la Confédération, \nSe...",498,N,19991206,4601,19999004,Ouverture de la législature,Ouverture de la législature
1,1,1,1,La séance d'aujourd'hui s'ouvre sous le signe ...,498,N,19991206,4601,19999004,Ouverture de la législature,Ouverture de la législature
2,2,53,6,Il appartenait au doyen d'âge de constituer le...,498,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,Conseil national.\nVérification des pouvoirs\n...
3,3,52,6,Das provisorische Büro hat an seiner Sitzung v...,273,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,Conseil national.\nVérification des pouvoirs\n...
4,4,51,6,Le Bureau provisoire de notre Assemblée a siég...,305,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,Conseil national.\nVérification des pouvoirs\n...
...,...,...,...,...,...,...,...,...,...,...,...
214764,189483,319686,60670,"Sehr geschätzter Herr Nationalrat Egger, Sie h...",4238,N,20230504,5120,20223026,Préserver les ressources écologiques en régula...,Motion Egger Mike.\nPréserver les ressources é...
214765,189484,319809,60674,La questione della mediatizzazione dei process...,4268,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,Motion Gysin Greta.\nMédiatisation des procès ...
214766,189485,319699,60674,"Frau Kollegin Gysin, Sie wollen eine Priorisie...",1122,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,Motion Gysin Greta.\nMédiatisation des procès ...
214767,189486,319807,60674,"Collega Fluri, sono molto consapevole del prob...",4268,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,Motion Gysin Greta.\nMédiatisation des procès ...


In [493]:
transcript_business_df = transcript_subject_df.merge(tags_df, on='BusinessNumber', how='left')
# drop rows where there is no tag
transcript_business_df = transcript_business_df.dropna(subset='Culture')
# set tags columns to int
transcript_business_df[tags_list] = transcript_business_df[tags_list].astype('int')
transcript_business_df

,index,ID,IdSubject,Text,PersonNumber,MeetingCouncilAbbreviation,MeetingDate,IdSession,BusinessNumber,Title,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
2,2,53,6,Il appartenait au doyen d'âge de constituer le...,498,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
3,3,52,6,Das provisorische Büro hat an seiner Sitzung v...,273,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
4,4,51,6,Le Bureau provisoire de notre Assemblée a siég...,305,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
5,5,50,6,"Le Bureau provisoire vous recommande, dans sa ...",498,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
6,6,48,6,Chaque conseiller national a communiqué s'il e...,498,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214764,189483,319686,60670,"Sehr geschätzter Herr Nationalrat Egger, Sie h...",4238,N,20230504,5120,20223026,Préserver les ressources écologiques en régula...,...,1,0,0,0,0,0,0,0,0,0
214765,189484,319809,60674,La questione della mediatizzazione dei process...,4268,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,...,0,0,0,0,1,1,0,0,0,0
214766,189485,319699,60674,"Frau Kollegin Gysin, Sie wollen eine Priorisie...",1122,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,...,0,0,0,0,1,1,0,0,0,0
214767,189486,319807,60674,"Collega Fluri, sono molto consapevole del prob...",4268,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,...,0,0,0,0,1,1,0,0,0,0


In [495]:
# get duplicated transcripts
# this happens when a Transcript is linked to a single Subject, which is linked to more than one Business
# for the majority of cases, a Transcript is linked to a single Subject, which is linked to a single Business

duplicates_df = transcript_business_df.loc[transcript_business_df.duplicated(subset=['ID'], keep=False)]
duplicates_df

,index,ID,IdSubject,Text,PersonNumber,MeetingCouncilAbbreviation,MeetingDate,IdSession,BusinessNumber,Title,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
6568,5505,8477,1642,"Ich werde die Fragen Heim, Bührer und Bezzola,...",134,N,20001204,4605,20005191,Vente aux enchères des licences UMTS,...,0,0,0,0,0,0,0,0,0,0
6569,5505,8477,1642,"Ich werde die Fragen Heim, Bührer und Bezzola,...",134,N,20001204,4605,20005196,Vente aux enchères des licences UMTS,...,0,0,0,0,0,0,0,0,0,0
6570,5505,8477,1642,"Ich werde die Fragen Heim, Bührer und Bezzola,...",134,N,20001204,4605,20005203,Vente aux enchères des licences UMTS,...,0,0,0,0,0,0,0,0,0,0
6571,5506,8476,1642,Die aktuellen Ereignisse seit letzter Woche ha...,399,N,20001204,4605,20005191,Vente aux enchères des licences UMTS,...,0,0,0,0,0,0,0,0,0,0
6572,5506,8476,1642,Die aktuellen Ereignisse seit letzter Woche ha...,399,N,20001204,4605,20005196,Vente aux enchères des licences UMTS,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214758,189480,319696,60673,"Die Frage, wie man mit Personen umgeht, die de...",4197,N,20230504,5120,20214536,Réintroduire dans la LAVI la possibilité d'ind...,...,0,0,0,0,0,1,0,0,0,0
214759,189481,319720,60673,Il est clair et juste que les victimes d'infra...,4238,N,20230504,5120,20214533,Réintroduire dans la LAVI la possibilité d'ind...,...,0,0,0,0,0,1,0,0,0,0
214760,189481,319720,60673,Il est clair et juste que les victimes d'infra...,4238,N,20230504,5120,20214534,Réintroduire dans la LAVI la possibilité d'ind...,...,0,0,0,0,0,1,0,0,0,0
214761,189481,319720,60673,Il est clair et juste que les victimes d'infra...,4238,N,20230504,5120,20214535,Réintroduire dans la LAVI la possibilité d'ind...,...,0,0,0,0,0,1,0,0,0,0


In [496]:
# group duplicates by transcript id, then calculate the mean of the topics by group
# if the mean of a topic for a transcript is lower than 1, then it means that not all transcripts in that group share that topic
# for example, 2 transcripts with topic "immigration" but one of them has topic "education" as well, 
# then only topic "immigration" has a mean of 1 and will be kept for these 2 transcripts

mean_df = duplicates_df.groupby('ID')[tags_list].mean()

for col in mean_df.columns:
    mean_df[col] = mean_df[col].apply(lambda x: 0 if x<1 else x)

mean_df = mean_df.astype('int')
mean_df.index.name = None

mean_df

,Éducation,Droit,Parlement,Économie,Politique internationale,Finances,Transports,Environnement,Santé,Politique européenne,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
8471,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8472,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8473,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8474,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8475,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319583,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
319587,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
319696,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
319720,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [497]:
mean_duplicates_df = duplicates_df[['ID']].merge(mean_df, right_index=True, left_on='ID', how='left')
mean_duplicates_df = mean_duplicates_df.drop(columns='ID')
mean_duplicates_df

,Éducation,Droit,Parlement,Économie,Politique internationale,Finances,Transports,Environnement,Santé,Politique européenne,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
6568,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6569,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6570,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6571,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6572,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214758,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
214759,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
214760,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
214761,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [498]:
transcript_processed_df = transcript_business_df.copy()
transcript_processed_df.loc[mean_duplicates_df.index, mean_duplicates_df.columns] = mean_duplicates_df
transcript_processed_df = transcript_processed_df.drop_duplicates(subset='ID')
transcript_processed_df

,index,ID,IdSubject,Text,PersonNumber,MeetingCouncilAbbreviation,MeetingDate,IdSession,BusinessNumber,Title,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
2,2,53,6,Il appartenait au doyen d'âge de constituer le...,498,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
3,3,52,6,Das provisorische Büro hat an seiner Sitzung v...,273,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
4,4,51,6,Le Bureau provisoire de notre Assemblée a siég...,305,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
5,5,50,6,"Le Bureau provisoire vous recommande, dans sa ...",498,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
6,6,48,6,Chaque conseiller national a communiqué s'il e...,498,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214764,189483,319686,60670,"Sehr geschätzter Herr Nationalrat Egger, Sie h...",4238,N,20230504,5120,20223026,Préserver les ressources écologiques en régula...,...,1,0,0,0,0,0,0,0,0,0
214765,189484,319809,60674,La questione della mediatizzazione dei process...,4268,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,...,0,0,0,0,1,1,0,0,0,0
214766,189485,319699,60674,"Frau Kollegin Gysin, Sie wollen eine Priorisie...",1122,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,...,0,0,0,0,1,1,0,0,0,0
214767,189486,319807,60674,"Collega Fluri, sono molto consapevole del prob...",4268,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,...,0,0,0,0,1,1,0,0,0,0


In [499]:
# drop rows without tag (meaning rows where sum of all tag columns = 0)
index_to_drop = transcript_processed_df[tags_list].sum(axis=1).loc[lambda x: x==0].index
print(len(index_to_drop))
transcript_processed_df = transcript_processed_df.drop(index_to_drop)
transcript_processed_df

673


,index,ID,IdSubject,Text,PersonNumber,MeetingCouncilAbbreviation,MeetingDate,IdSession,BusinessNumber,Title,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
2,2,53,6,Il appartenait au doyen d'âge de constituer le...,498,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
3,3,52,6,Das provisorische Büro hat an seiner Sitzung v...,273,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
4,4,51,6,Le Bureau provisoire de notre Assemblée a siég...,305,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
5,5,50,6,"Le Bureau provisoire vous recommande, dans sa ...",498,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
6,6,48,6,Chaque conseiller national a communiqué s'il e...,498,N,19991206,4601,19990083,Conseil national. Vérification des pouvoirs,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214764,189483,319686,60670,"Sehr geschätzter Herr Nationalrat Egger, Sie h...",4238,N,20230504,5120,20223026,Préserver les ressources écologiques en régula...,...,1,0,0,0,0,0,0,0,0,0
214765,189484,319809,60674,La questione della mediatizzazione dei process...,4268,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,...,0,0,0,0,1,1,0,0,0,0
214766,189485,319699,60674,"Frau Kollegin Gysin, Sie wollen eine Priorisie...",1122,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,...,0,0,0,0,1,1,0,0,0,0
214767,189486,319807,60674,"Collega Fluri, sono molto consapevole del prob...",4268,N,20230504,5120,20223083,Médiatisation des procès pénaux. La protection...,...,0,0,0,0,1,1,0,0,0,0


In [517]:
transcript_tags_df = transcript_processed_df.set_index('ID').sort_index()
transcript_tags_df = transcript_tags_df[tags_list]
transcript_tags_df.index.name = 'transcript_id'
transcript_tags_df

,Éducation,Droit,Parlement,Économie,Politique internationale,Finances,Transports,Environnement,Santé,Politique européenne,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
transcript_id,,,,,,,,,,,,,,,,,,,,,
48,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319800,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
319801,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
319802,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [518]:
transcript_tags_df.to_csv('data/transcripts_tags.csv', encoding='utf-8')

In [516]:
# number of topics by transcript in percent
round(transcript_processed_df[tags_list].sum(axis=1).sort_values().value_counts(normalize=True)*100,2)

1    61.01
2    24.91
3    10.00
4     3.03
5     0.52
7     0.30
9     0.16
6     0.05
8     0.00
Name: proportion, dtype: float64

## Tag exploration

In [255]:
with open('data/transcripts_tags.csv', encoding='utf-8') as file:
    tags_df = pd.read_csv(file)
    
tags_df

,transcript_id,Éducation,Droit,Parlement,Économie,Politique internationale,Finances,Transports,Environnement,Santé,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
0,48,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,50,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,51,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,52,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,53,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172433,319800,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
172434,319801,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
172435,319802,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
172436,319807,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0


In [124]:
with open('data/persons.csv', encoding='utf-8') as file:
    persons_df = pd.read_csv(file).drop(columns='Unnamed: 0')
    persons_df['PartyAbbreviation'] = persons_df['PartyAbbreviation'].replace({'PDC': 'M-E'})
    
persons_df

,PersonNumber,FirstName,LastName,GenderAsString,ParlGroupName,PartyName,PartyAbbreviation,DateJoining,DateLeaving,DateElection,DateOath,DateResignation,DateOfBirth,DateOfDeath
0,9,Ruedi,Baumann,m,Groupe des VERT-E-S,Les VERT-E-S suisses,VERT-E-S,1995-12-04T00:00:00,2003-11-30T00:00:00,1995-12-04T00:00:00,1995-12-04T00:00:00,2003-11-30T00:00:00,1947-11-11T00:00:00,NaN
1,12,Christine,Beerli,f,Groupe libéral-radical,PLR.Les Libéraux-Radicaux,PLR,2002-11-25T00:00:00,2003-11-30T00:00:00,1995-12-04T00:00:00,1995-12-04T00:00:00,2003-11-30T00:00:00,1953-03-26T00:00:00,NaN
2,14,Duri,Bezzola,m,Groupe libéral-radical,PLR.Les Libéraux-Radicaux,PLR,2003-12-01T00:00:00,2007-03-04T00:00:00,2003-10-19T00:00:00,2003-12-01T00:00:00,2007-03-04T00:00:00,1942-06-23T00:00:00,NaN
3,15,Max,Binder,m,Groupe de l'Union démocratique du Centre,Union Démocratique du Centre,UDC,2011-12-05T00:00:00,2015-11-29T00:00:00,2011-10-23T00:00:00,2011-12-05T00:00:00,2015-11-29T00:00:00,1947-11-26T00:00:00,NaN
4,21,Christoph,Blocher,m,Groupe de l'Union démocratique du Centre,Union Démocratique du Centre,UDC,2011-12-05T00:00:00,2014-05-31T00:00:00,2011-10-23T00:00:00,2011-12-05T00:00:00,2014-05-31T00:00:00,1940-10-11T00:00:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705,4329,Daniel,Ruch,m,Groupe libéral-radical,PLR.Les Libéraux-Radicaux,PLR,2022-06-13T00:00:00,NaN,2019-10-20T00:00:00,2022-06-13T00:00:00,NaN,1963-03-20T00:00:00,NaN
706,4330,Alexandre,Berthoud,m,Groupe libéral-radical,PLR.Les Libéraux-Radicaux,PLR,2022-06-13T00:00:00,NaN,2019-10-20T00:00:00,2022-06-13T00:00:00,NaN,1977-06-29T00:00:00,NaN
707,4331,Marc,Jost,m,Le Groupe du Centre. Le Centre. PEV.,Parti évangélique suisse,PEV,2022-11-28T00:00:00,NaN,2019-10-20T00:00:00,2022-11-28T00:00:00,NaN,1974-02-06T00:00:00,NaN
708,4332,Mathilde,Crevoisier Crelier,f,Groupe socialiste,Parti socialiste suisse,PSS,2022-12-15T00:00:00,NaN,2019-10-20T00:00:00,2022-12-15T00:00:00,NaN,1980-01-05T00:00:00,NaN


In [125]:
# get file paths

directory = 'data/transcripts'
filepaths_list = []
transcripts_list = []

# iterate over files in that directory
for filename in os.listdir(directory):
    file = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(file) and file.endswith('.csv'):
        filepaths_list.append(file)

# sort transcripts chronologically 
filepaths_list.sort()
print(len(filepaths_list))

117


In [235]:
transcript_df = pd.DataFrame()

for filepath in tqdm(filepaths_list[:]):
    with open(filepath, encoding='utf-8') as file:
        session_df = pd.read_csv(file).drop(columns='Unnamed: 0')
        if session_df.iloc[0]['IdSession'] > 4700:
            transcript_df = pd.concat([transcript_df, session_df])

transcript_df = transcript_df.reset_index(drop=True)
transcript_df['legislature'] = transcript_df['IdSession'].apply(lambda x: str(x)[:2])
transcript_df['legislature'] = transcript_df['legislature'].astype('int')
transcript_df

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 117/117 [00:05<00:00, 22.52it/s]


,ID,IdSubject,PersonNumber,Text,MeetingCouncilAbbreviation,MeetingDate,IdSession,SpeakerFunction,CouncilId,Start,End,Function,LanguageOfText,legislature
0,38761,6585,21,"Präsident (Blocher Christoph, Alterspräsident)...",N,20031201,4701,Mit-M,1.0,2003-12-01T14:31:16,2003-12-01T14:32:35,NaN,DE,47
1,38759,6585,21,"Präsident (Blocher Christoph, Alterspräsident)...",N,20031201,4701,Mit-M,1.0,2003-12-01T14:40:37,2003-12-01T14:53:15,Mit-M,DE,47
2,38758,6585,1131,Wir alle wissen: 1989 fiel in Berlin die Mauer...,N,20031201,4701,Mit-F,1.0,2003-12-01T14:54:06,2003-12-01T15:04:59,Mit-F,DE,47
3,38734,6581,21,"Präsident (Blocher Christoph, Alterspräsident)...",N,20031201,4701,Mit-M,1.0,2003-12-01T15:04:59,2003-12-01T15:11:33,Mit-M,DE,47
4,38732,6581,21,"Präsident (Blocher Christoph, Alterspräsident)...",N,20031201,4701,Mit-M,1.0,2003-12-01T15:21:38,2003-12-01T15:23:11,Mit-M,DE,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161583,319686,60670,4238,"Sehr geschätzter Herr Nationalrat Egger, Sie h...",N,20230504,5120,BR-F,99.0,2023-05-04T17:04:38,2023-05-04T17:06:20,BR-F,DE,51
161584,319809,60674,4268,La questione della mediatizzazione dei process...,N,20230504,5120,Mit-F,1.0,2023-05-04T17:06:39,2023-05-04T17:11:27,Mit-F,IT,51
161585,319699,60674,1122,"Frau Kollegin Gysin, Sie wollen eine Priorisie...",N,20230504,5120,Mit-M,1.0,2023-05-04T17:11:29,2023-05-04T17:11:46,Mit-M,DE,51
161586,319807,60674,4268,"Collega Fluri, sono molto consapevole del prob...",N,20230504,5120,Mit-F,1.0,2023-05-04T17:11:46,2023-05-04T17:12:44,Mit-F,IT,51


In [236]:
# get the length (number of characters) of each transcript
transcript_df['length'] = transcript_df['Text'].apply(lambda x: len(x))

# reduce the length by dividing it by 1000 (max length = +43k, max reduced length = 43)
transcript_df['length_norm'] = transcript_df['length'] / 1000

# normalize the reduced length by applying exponential formula (range between 0 and 1)
# length < 500 => weigth < 0.4 // length < 1000 => weigth < 0.6 // length > 3000 => weigth > 0.95
transcript_df['weight'] = transcript_df['length_norm'].apply(lambda x: round(1 - math.exp(-x), 2))

transcript_df

,ID,IdSubject,PersonNumber,Text,MeetingCouncilAbbreviation,MeetingDate,IdSession,SpeakerFunction,CouncilId,Start,End,Function,LanguageOfText,legislature,length,length_norm,weight
0,38761,6585,21,"Präsident (Blocher Christoph, Alterspräsident)...",N,20031201,4701,Mit-M,1.0,2003-12-01T14:31:16,2003-12-01T14:32:35,NaN,DE,47,269,0.269,0.24
1,38759,6585,21,"Präsident (Blocher Christoph, Alterspräsident)...",N,20031201,4701,Mit-M,1.0,2003-12-01T14:40:37,2003-12-01T14:53:15,Mit-M,DE,47,8883,8.883,1.00
2,38758,6585,1131,Wir alle wissen: 1989 fiel in Berlin die Mauer...,N,20031201,4701,Mit-F,1.0,2003-12-01T14:54:06,2003-12-01T15:04:59,Mit-F,DE,47,7752,7.752,1.00
3,38734,6581,21,"Präsident (Blocher Christoph, Alterspräsident)...",N,20031201,4701,Mit-M,1.0,2003-12-01T15:04:59,2003-12-01T15:11:33,Mit-M,DE,47,7822,7.822,1.00
4,38732,6581,21,"Präsident (Blocher Christoph, Alterspräsident)...",N,20031201,4701,Mit-M,1.0,2003-12-01T15:21:38,2003-12-01T15:23:11,Mit-M,DE,47,366,0.366,0.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161583,319686,60670,4238,"Sehr geschätzter Herr Nationalrat Egger, Sie h...",N,20230504,5120,BR-F,99.0,2023-05-04T17:04:38,2023-05-04T17:06:20,BR-F,DE,51,1656,1.656,0.81
161584,319809,60674,4268,La questione della mediatizzazione dei process...,N,20230504,5120,Mit-F,1.0,2023-05-04T17:06:39,2023-05-04T17:11:27,Mit-F,IT,51,4506,4.506,0.99
161585,319699,60674,1122,"Frau Kollegin Gysin, Sie wollen eine Priorisie...",N,20230504,5120,Mit-M,1.0,2023-05-04T17:11:29,2023-05-04T17:11:46,Mit-M,DE,51,230,0.230,0.21
161586,319807,60674,4268,"Collega Fluri, sono molto consapevole del prob...",N,20230504,5120,Mit-F,1.0,2023-05-04T17:11:46,2023-05-04T17:12:44,Mit-F,IT,51,980,0.980,0.62


In [250]:
# remove some speaker functions from transcripts (Bundesrat, Bundespräsident, Vize-Präsident Bundesrat, Bundeskanzler, Ratspräsident)
# NB: legislature 46 has 19k transcripts without speaker function, so it was dropped

function_to_drop = [
    'BR-M',
    'BR-F',
    'VPBR-M',
    'VPBR-F',
    'BPR-M',
    'BPR-F',
    'BK-M', 
    'BK-F', 
    'P-M', 
    'P-F',
]
transcript_df = transcript_df.loc[~transcript_df['SpeakerFunction'].isin(function_to_drop)]
print('transcripts without function:', transcript_df['SpeakerFunction'].isna().sum())
print('number of transcripts:', len(transcript_df))
transcript_df.query('legislature > 40')['SpeakerFunction'].value_counts()

transcripts without function: 0
number of transcripts: 132164


SpeakerFunction
Mit-M     90947
Mit-F     32902
1VP-M      4627
1VP-F      1565
2VP-M      1407
2VP-F       715
Mit-F         1
Name: count, dtype: int64

In [251]:
max_length = transcript_df.length.max()
length_list = list(transcript_df['length'])
distrib = dict()
divider = 1000

for i in range(max_length//divider + 1):
    distrib[i] = 0
    
for i in length_list:
    distrib[i//divider] += 1
    
distrib

{0: 45491,
 1: 22047,
 2: 20773,
 3: 18928,
 4: 12337,
 5: 5519,
 6: 2792,
 7: 1652,
 8: 1045,
 9: 609,
 10: 357,
 11: 209,
 12: 137,
 13: 90,
 14: 49,
 15: 28,
 16: 29,
 17: 11,
 18: 16,
 19: 9,
 20: 7,
 21: 4,
 22: 3,
 23: 9,
 24: 4,
 25: 1,
 26: 1,
 27: 2,
 28: 1,
 29: 0,
 30: 0,
 31: 1,
 32: 1,
 33: 0,
 34: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 0,
 40: 1,
 41: 0,
 42: 1}

In [252]:
persons_col = ['PersonNumber', 'FirstName', 'LastName', 'GenderAsString', 'PartyAbbreviation', 'DateJoining', 'DateLeaving']
transcript_person_df = transcript_df.merge(persons_df[persons_col], on='PersonNumber', how='left')
transcript_person_df

,ID,IdSubject,PersonNumber,Text,MeetingCouncilAbbreviation,MeetingDate,IdSession,SpeakerFunction,CouncilId,Start,...,legislature,length,length_norm,weight,FirstName,LastName,GenderAsString,PartyAbbreviation,DateJoining,DateLeaving
0,38761,6585,21,"Präsident (Blocher Christoph, Alterspräsident)...",N,20031201,4701,Mit-M,1.0,2003-12-01T14:31:16,...,47,269,0.269,0.24,Christoph,Blocher,m,UDC,2011-12-05T00:00:00,2014-05-31T00:00:00
1,38759,6585,21,"Präsident (Blocher Christoph, Alterspräsident)...",N,20031201,4701,Mit-M,1.0,2003-12-01T14:40:37,...,47,8883,8.883,1.00,Christoph,Blocher,m,UDC,2011-12-05T00:00:00,2014-05-31T00:00:00
2,38758,6585,1131,Wir alle wissen: 1989 fiel in Berlin die Mauer...,N,20031201,4701,Mit-F,1.0,2003-12-01T14:54:06,...,47,7752,7.752,1.00,Evi,Allemann,f,PSS,2015-11-30T00:00:00,2018-05-28T00:00:00
3,38734,6581,21,"Präsident (Blocher Christoph, Alterspräsident)...",N,20031201,4701,Mit-M,1.0,2003-12-01T15:04:59,...,47,7822,7.822,1.00,Christoph,Blocher,m,UDC,2011-12-05T00:00:00,2014-05-31T00:00:00
4,38732,6581,21,"Präsident (Blocher Christoph, Alterspräsident)...",N,20031201,4701,Mit-M,1.0,2003-12-01T15:21:38,...,47,366,0.366,0.31,Christoph,Blocher,m,UDC,2011-12-05T00:00:00,2014-05-31T00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132159,319696,60673,4197,"Die Frage, wie man mit Personen umgeht, die de...",N,20230504,5120,Mit-F,1.0,2023-05-04T16:52:10,...,51,1325,1.325,0.73,Min Li,Marti,f,PSS,2019-12-02T00:00:00,NaN
132160,319692,60670,4229,Der Bundesrat soll mit meiner Motion beauftrag...,N,20230504,5120,Mit-M,1.0,2023-05-04T16:58:04,...,51,4768,4.768,0.99,Mike,Egger,m,UDC,2019-12-02T00:00:00,NaN
132161,319809,60674,4268,La questione della mediatizzazione dei process...,N,20230504,5120,Mit-F,1.0,2023-05-04T17:06:39,...,51,4506,4.506,0.99,Greta,Gysin,f,VERT-E-S,2019-12-02T00:00:00,NaN
132162,319699,60674,1122,"Frau Kollegin Gysin, Sie wollen eine Priorisie...",N,20230504,5120,Mit-M,1.0,2023-05-04T17:11:29,...,51,230,0.230,0.21,Kurt,Fluri,m,PLR,2019-12-02T00:00:00,NaN


In [253]:
transcript_tagged_df = transcript_person_df.merge(tags_df, left_on='ID', right_on='transcript_id', how='left')
transcript_tagged_df = transcript_tagged_df.dropna(subset='Culture')
transcript_tagged_df

,ID,IdSubject,PersonNumber,Text,MeetingCouncilAbbreviation,MeetingDate,IdSession,SpeakerFunction,CouncilId,Start,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
3,38734,6581,21,"Präsident (Blocher Christoph, Alterspräsident)...",N,20031201,4701,Mit-M,1.0,2003-12-01T15:04:59,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,38732,6581,21,"Präsident (Blocher Christoph, Alterspräsident)...",N,20031201,4701,Mit-M,1.0,2003-12-01T15:21:38,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,38736,6582,273,"Die Stiftung Bibliomedia Schweiz, 1920 als Sti...",N,20031201,4701,Mit-F,1.0,2003-12-01T17:44:08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,38737,6582,511,"La fondation Bibliomedia suisse, créée en 1920...",N,20031201,4701,Mit-F,1.0,2003-12-01T17:50:47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,38738,6582,311,"Le président (Christen Yves, R, VD): Nous trai...",N,20031201,4701,Mit-M,1.0,2003-12-01T17:59:30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132159,319696,60673,4197,"Die Frage, wie man mit Personen umgeht, die de...",N,20230504,5120,Mit-F,1.0,2023-05-04T16:52:10,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
132160,319692,60670,4229,Der Bundesrat soll mit meiner Motion beauftrag...,N,20230504,5120,Mit-M,1.0,2023-05-04T16:58:04,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
132161,319809,60674,4268,La questione della mediatizzazione dei process...,N,20230504,5120,Mit-F,1.0,2023-05-04T17:06:39,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
132162,319699,60674,1122,"Frau Kollegin Gysin, Sie wollen eine Priorisie...",N,20230504,5120,Mit-M,1.0,2023-05-04T17:11:29,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [256]:
transcript_filtered_df = transcript_tagged_df.copy()
tags_list = list(tags_df.columns)
tags_list.remove('transcript_id')

# join droit droit civil, droit pénal, droit
transcript_filtered_df['Droit'] = transcript_tagged_df['Droit'] + transcript_tagged_df['Droit civil'] + transcript_tagged_df['Droit pénal']
transcript_filtered_df['Droit'] = transcript_filtered_df['Droit'].replace({2: 1, 3: 1})
transcript_filtered_df = transcript_filtered_df.drop(columns=['Droit civil', 'Droit pénal'])

# join politique inter, politique euro
#transcript_filtered_df['Politique internationale'] = transcript_tagged_df['Politique européenne'] + transcript_tagged_df['Politique internationale']
#transcript_filtered_df['Politique internationale'].replace({2: 1})
#transcript_filtered_df = transcript_filtered_df.drop(columns=['Politique européenne'])

# update tags list
try:
    #tags_list.remove('Politique européenne')
    tags_list.remove('Droit civil')
    tags_list.remove('Droit pénal')
except ValueError:
    pass

transcript_filtered_df

,ID,IdSubject,PersonNumber,Text,MeetingCouncilAbbreviation,MeetingDate,IdSession,SpeakerFunction,CouncilId,Start,...,Énergie,Aménagement du territoire et logement,Politique migratoire,Culture,Emploi et travail,Droits de l'homme,Droit international,Fiscalité,Protection sociale,Justice
3,38734,6581,21,"Präsident (Blocher Christoph, Alterspräsident)...",N,20031201,4701,Mit-M,1.0,2003-12-01T15:04:59,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,38732,6581,21,"Präsident (Blocher Christoph, Alterspräsident)...",N,20031201,4701,Mit-M,1.0,2003-12-01T15:21:38,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,38736,6582,273,"Die Stiftung Bibliomedia Schweiz, 1920 als Sti...",N,20031201,4701,Mit-F,1.0,2003-12-01T17:44:08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,38737,6582,511,"La fondation Bibliomedia suisse, créée en 1920...",N,20031201,4701,Mit-F,1.0,2003-12-01T17:50:47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,38738,6582,311,"Le président (Christen Yves, R, VD): Nous trai...",N,20031201,4701,Mit-M,1.0,2003-12-01T17:59:30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132159,319696,60673,4197,"Die Frage, wie man mit Personen umgeht, die de...",N,20230504,5120,Mit-F,1.0,2023-05-04T16:52:10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
132160,319692,60670,4229,Der Bundesrat soll mit meiner Motion beauftrag...,N,20230504,5120,Mit-M,1.0,2023-05-04T16:58:04,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
132161,319809,60674,4268,La questione della mediatizzazione dei process...,N,20230504,5120,Mit-F,1.0,2023-05-04T17:06:39,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
132162,319699,60674,1122,"Frau Kollegin Gysin, Sie wollen eine Priorisie...",N,20230504,5120,Mit-M,1.0,2023-05-04T17:11:29,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [257]:
main_party = [
    'M-E',
    'PLR',
    'PSS',
    'UDC',
    'VERT-E-S',
    'pvl',
]

In [258]:
transcript_filtered_47 = transcript_filtered_df.loc[transcript_filtered_df['legislature'] == 47]
transcript_filtered_48 = transcript_filtered_df.loc[transcript_filtered_df['legislature'] == 48]
transcript_filtered_49 = transcript_filtered_df.loc[transcript_filtered_df['legislature'] == 49]
transcript_filtered_50 = transcript_filtered_df.loc[transcript_filtered_df['legislature'] == 50]
transcript_filtered_51 = transcript_filtered_df.loc[transcript_filtered_df['legislature'] == 51]

In [269]:
group = {'gender': 'GenderAsString', 'party': 'PartyAbbreviation'}
ratio_by_party = True

tag_by_group = transcript_filtered_50.groupby(group['party'])[tags_list].sum()
if ratio_by_party:
    tag_by_group = tag_by_group.loc[:,:].div(tag_by_group.sum(axis=1), axis=0)
else:    
    tag_by_group = tag_by_group.loc[:,:].div(tag_by_group.sum(axis=0), axis=1)

tag_by_group_50 = tag_by_group

In [270]:
tag_by_group_50.groupby('PartyAbbreviation')[tags_list].mean().loc[main_party].T.style.background_gradient(cmap='YlGnBu', axis=0)

PartyAbbreviation,M-E,PLR,PSS,UDC,VERT-E-S,pvl
Éducation,0.015471,0.012657,0.011950,0.012362,0.009082,0.005432
Droit,0.045516,0.061492,0.058556,0.058066,0.065142,0.061380
Parlement,0.015919,0.021505,0.017607,0.018456,0.016599,0.012493
Économie,0.083857,0.092630,0.084927,0.064769,0.094269,0.071700
Politique internationale,0.062892,0.072133,0.068276,0.063811,0.044159,0.049430
Finances,0.108408,0.099686,0.111456,0.110124,0.087692,0.090168
Transports,0.027466,0.036738,0.027406,0.027335,0.027560,0.038566
Environnement,0.048318,0.036962,0.039356,0.052146,0.087692,0.076046
Santé,0.049888,0.043795,0.058955,0.043789,0.056687,0.040196
Politique européenne,0.034081,0.045475,0.036170,0.063289,0.031319,0.026616


## Verification

In [461]:
print(transcript_business_df.loc[7521]['ID'])

267141


In [463]:
transcript_id = 267141
transcript_business_df.query('ID == ' + str(transcript_id))

,index,ID,IdSubject,Text,PersonNumber,MeetingCouncilAbbreviation,MeetingDate,IdSession,BusinessNumber,Title,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
7521,6867,267141,49812,Das ist der zweite Teil meiner gestrigen Motio...,4172,S,20200910,5105,20200058,Loi fédérale sur les bases légales des ordonna...,...,1,1,1,0,0,1,0,0,1,0


In [432]:
sample_df = transcript_business_df.loc[transcript_business_df['ID'] == transcript_id][tags_list]
sample_df

,Éducation,Droit,Parlement,Économie,Politique internationale,Finances,Transports,Environnement,Santé,Politique européenne,...,Politique migratoire,Culture,Droit civil,Emploi et travail,Droits de l'homme,Droit pénal,Droit international,Fiscalité,Protection sociale,Justice
1181,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1182,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1183,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1184,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1185,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [433]:
sample_df[sample_df.columns[sample_df.sum() > 0]]

,Finances,Santé,Questions sociales
1181,1,1,0
1182,1,1,0
1183,1,1,0
1184,1,1,0
1185,0,1,1


In [434]:
sample_df[sample_df.columns[sample_df.sum() == len(sample_df)]]

,Santé
1181,1
1182,1
1183,1
1184,1
1185,1
